# Imports

In [1]:
import pandas as pd
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
from sklearn.preprocessing import MinMaxScaler

# Helper Functions 

## Helper Functions for cleaning data from Ballchasing.com

In [2]:
def get_totals(df):
    # Select only the columns that contain the total counts for each statistic over the course of the season.
    total = df[[
    'team name', 'player name', 'games', 'wins', 'score', 'goals', 'assists', 'saves', 'shots', 'shots conceded', 'goals conceded',
    'goals conceded while last defender', 'amount collected', 'amount collected big pads', 'amount collected small pads',  
    'count collected big pads', 'count collected small pads','amount stolen', 'amount stolen big pads', 'amount stolen small pads',
    'count stolen big pads', 'count stolen small pads', '0 boost time', '100 boost time', 'amount used while supersonic',
    'amount overfill total','amount overfill stolen', 'total distance', 'time slow speed', 'time boost speed', 'time supersonic speed',
    'time on ground', 'time low in air', 'time high in air', 'time powerslide','count powerslide','time most back', 'time most forward',
    'time in front of ball','time behind ball', 'time defensive half', 'time offensive half', 'time defensive third', 'time neutral third',
    'time offensive third','demos inflicted', 'demos taken'
    ]]
    # Lowercase all playernames for consistency.
    total['player name'] = total["player name"].str.lower() 
    # Since we only have a dataframe of total statistics, we can group by player to account for BallChasing.com accidently mismanaging 
    # team and player relationships. This is acting as a merge between rows where playernames match but the teams names didnt. 
    cleaned_data = total.groupby(['player name']).sum().reset_index()
    return cleaned_data

In [3]:
def average_player_statistics(cleaned_data):
    # In order to round the data to the nearest thousandth, we need to remove all columns without int datatypes. I divide the total
    # statistics for every player by the total number of (recorded) games that they have played and uploaded replays. 
    per_game_stats = cleaned_data.iloc[:,2:].div(cleaned_data.games, axis=0).round(4)
    # This is theseries that contains all columns without int datatypes, in other words all of the players. 
    players = cleaned_data.iloc[:,:2]
    # Then we merge these back together for the final dataframe.
    final_df = pd.concat([players, per_game_stats], axis=1)
    # return df so we can 
    return final_df

In [4]:
def remove_player_statistics(df, lst):
    # Provide as input a list of player names to be drop from this dataframe. 
    return df[~df['player name'].isin(lst)]

In [5]:
def patch_duplicates(df, dictionary):
    all_players = [list(dictionary.keys()), list(dictionary.values())]
    flattened = [item for sublist in all_players for item in sublist]
    # data = df.set_index('player name', drop = True, verify_integrity = True)
    data = df.set_index('player name', drop = True)
    # Remove players 
    patched_data = data.loc[flattened]
    patched_data = patched_data.rename(index=dictionary)
    patched_data = patched_data.reset_index(drop = False)\
        .groupby(['player name']).sum()\
            .reset_index(drop=False)
    # Removed players that we combined so we can recombine them
    removed_duplicates = remove_player_statistics(df, flattened).reset_index(drop = True)
    # Combine the two
    patched_dataframe = pd.concat([removed_duplicates, patched_data])
    return patched_dataframe.reset_index(drop=True)

## Helper Functions for generating player standings 

Current Season = TRL Winter 2021 Tournament

Win Rate

Offensive Rating
- score per game + shots per game + assists per game + shooting percentage + time most forward per game + time offensive half per game + time neurtal third per game + time offensive third per game

Defensive Rating
- score per game + saves per game + goals conceded per game + goals conceded while last defender per game + time most back per game + time defensive half per game + time neurtal third per game + time defensive third per game

Aggression
- shots per game + average speed per game + amount stolen per game + amount stolen big pads per game + amount stolen small pads per game + time offensive half per game + demos inflicted per game + demos taken per game + time powerslide per game + avg powerslide time per game + count powerslide per game

Speed
- bpm per game + avg boost amount per game + time supersonic per game +total distance per game + avg speed per game + average amount used while supersonic per game + amount collected per game + amount collected big pads per game + amount collected small pads per game + 0 boost time per game + 100 boost time per game + amount used while supersonic per game + time slow speed per game + time boost speed per game + time supersonic speed per game

In [6]:
def offensive_stats(df):
    # Select all columns with relevant data 
    Offensive = df[[
        'player name', 'score', 'shots', 'assists',
        'time most forward','time offensive half',
        'time neutral third','time offensive third'
        ]]
    # Create a new column with the calculated offensive rating value based on original column data calculations
    Offensive['Offensive Rating'] = .05*Offensive['score'] + .2*Offensive['shots'] \
                                + .05*Offensive['assists'] + .05*Offensive['time most forward'] + .05*Offensive['time neutral third'] \
                                + .1*Offensive['time offensive third'] 
    # The we use the MinMaxScaler from SKLearn to normalize all of the values and to generate ranks. 
    scaler = MinMaxScaler()
    Offensive['Offensive Rating']  = scaler.fit_transform(Offensive[['Offensive Rating']])
    # Return the DataFrame containing [player name, offensive rating] 
    # Columns in sorted order with respect to Offensive rating  
    return Offensive.sort_values('Offensive Rating',ascending=False)[['player name', 'Offensive Rating']]

In [7]:
def defensive_stats(df):
    # Select all columns with relevant data 
    Defensive = df[[
        'player name', 'score', 'saves','goals conceded','goals conceded while last defender',
        'time most back','time defensive half','time neutral third','time defensive third'
        ]]
    # Create a new column with the calculated defensive rating value based on original column data calculations
    Defensive['Defensive Rating'] = .05* Defensive['score']  + .2*Defensive['saves'] + .05*Defensive['goals conceded'] +\
                                    .1*Defensive['goals conceded while last defender']+ .1*Defensive['time most back'] +\
                                    .15*Defensive['time defensive half'] + .1 * Defensive['time neutral third'] +\
                                    .05 * Defensive['time defensive third']
    # The we use the MinMaxScaler from SKLearn to normalize all of the values and to generate ranks. 
    scaler = MinMaxScaler()
    Defensive['Defensive Rating']  = scaler.fit_transform(Defensive[['Defensive Rating']])
    # Return the DataFrame containing [player name, defensive rating] 
    # Columns in sorted order with respect to Offensive rating  
    return Defensive.sort_values('Defensive Rating',ascending=False)[['player name', 'Defensive Rating']]

In [8]:
def aggression_stats(df):
    # Select all columns with relevant data 
    Aggression = df[[
        'player name', 'shots', 'amount stolen', 'amount stolen big pads', 'amount stolen small pads',
        'time offensive half','demos inflicted','demos taken','time powerslide','count powerslide'
        ]]
    # Create a new column with the calculated defensive rating value based on original column data calculations
    Aggression['Aggression Rating'] = .2*Aggression['shots'] +.025*Aggression['amount stolen']+ .05*Aggression['amount stolen big pads']+\
                                        .025*Aggression['amount stolen small pads'] + .075*Aggression['time offensive half'] +\
                                        .2*Aggression['demos inflicted'] + .025*Aggression['demos taken'] +\
                                        .05*Aggression['time powerslide'] + .15*Aggression['count powerslide']
    # The we use the MinMaxScaler from SKLearn to normalize all of the values and to generate ranks.
    scaler = MinMaxScaler()
    Aggression['Aggression Rating']  = scaler.fit_transform(Aggression[['Aggression Rating']])
    # Return the DataFrame containing [player name, aggression rating] 
    # Columns in sorted order with respect to aggression rating  
    return Aggression.sort_values('Aggression Rating',ascending=False)[['player name', 'Aggression Rating']]

In [9]:
def speed_stats(df):
    # Select all columns with relevant data 
    Speed = df[[
        'player name','total distance','amount collected','amount collected big pads','amount collected small pads',
        '0 boost time','100 boost time','amount used while supersonic','time slow speed','time boost speed','time supersonic speed'
        ]]
    # Create a new column with the calculated defensive rating value based on original column data calculations
    Speed['Speed Rating'] = + 4*Speed['total distance']\
                        + (-3*Speed['amount used while supersonic']) + 1*Speed['amount collected']\
                        + 1*Speed['amount collected big pads'] + 2*Speed['amount collected small pads'] + (-3*Speed['0 boost time'])\
                        + 3*Speed['100 boost time'] + -1*Speed['time slow speed'] + 1*Speed['time boost speed']\
                        + 3*Speed['time supersonic speed']
    # The we use the MinMaxScaler from SKLearn to normalize all of the values and to generate ranks.
    scaler = MinMaxScaler()
    # Return the DataFrame containing [player name, speed rating] 
    # Columns in sorted order with respect to speed rating  
    Speed['Speed Rating'] = scaler.fit_transform(Speed[['Speed Rating']])
    return Speed.sort_values('Speed Rating',ascending=False)[['player name', 'Speed Rating']]

## Helper Functions to Generate Final Overall Standings DataFrame

In [10]:
def generate_overall_standings(df, offensive_stats, defensive_stats, aggression_stats, speed_stats):
    # Take name and win rate from cleaned dataframe
    final_df = df[['player name', 'wins']]
    # Add the offensive player ratings to final df by merging on playername.
    final_df = final_df.merge(offensive_stats, how='left')
    # Add the defensive player ratings to final df by merging on playername.
    final_df = final_df.merge(defensive_stats, how='left')
    # Add the aggression player ratings to final df by merging on playername.
    final_df = final_df.merge(aggression_stats, how='left')
    # Add the speed player ratings to final df by merging on playername.
    final_df = final_df.merge(speed_stats, how='left')

    # Formatting by reordering columns in a readable format. 
    overalls = final_df[['player name','wins','Offensive Rating', 'Defensive Rating', 'Aggression Rating', 'Speed Rating']]

    # Generate overall standings based on all generated stats
    overalls['Total Overall'] = .3*overalls['Offensive Rating'] + .2*overalls['Defensive Rating'] +\
                                .35*overalls['Aggression Rating'] + .15*overalls['Speed Rating']
    # Scaler for normalization                             
    scaler = MinMaxScaler()
    # Normalize
    overalls['Total Overall'] = scaler.fit_transform(overalls[['Total Overall']])
    # Sort values by total overall score
    total_overalls = overalls.sort_values('Total Overall',ascending=False)

    # Formatting for easier human readability
    total_overalls.reset_index(drop=True).round(3)
    total_overalls['Offensive Rating'] = total_overalls['Offensive Rating'] *100 
    total_overalls['Defensive Rating'] = total_overalls['Defensive Rating'] *100 
    total_overalls['Aggression Rating'] = total_overalls['Aggression Rating'] *100 
    total_overalls['Speed Rating'] = total_overalls['Speed Rating'] *100 
    total_overalls['Total Overall'] = total_overalls['Total Overall'] *100 

    # Display the final dataframe for people to inspect
    results = total_overalls[[
        'player name', 'wins', 'Total Overall',
        'Offensive Rating', 'Defensive Rating', 'Aggression Rating', 'Speed Rating']]\
            .round(2).reset_index(drop=True)\
                .rename(columns={
                    "player name": "participant",
                    'wins': 'win rate',
                    "Total Overall": "Overall",
                    'Offensive Rating': 'Offense',
                    'Defensive Rating': 'Defense',
                    'Aggression Rating':'Aggression',
                    'Speed Rating': 'Speed'})

    return results


# Load in Fall 2020 player data

In [91]:
fall_2020 = 'data\TRL_F20_playerdata_final.csv'
df = pd.read_csv(fall_2020)
fall_2020_totals = get_totals(df)
fall_2020_averages = average_player_statistics(fall_2020_totals)

# Load in Winter Player Data

In [11]:
winter_2021 = 'data\TRL_W21_playerdata.csv'
winter_2021 = pd.read_csv(winter_2021)

# Calculate Overalls 

In [12]:
winter_2021_totals = get_totals(winter_2021)

name_changes = {'invincibleblaze':'invincible','nsdlakers4': 'shaunch'}
# Add a helper function to combine players who have changed their steam/user name
patched_totals = patch_duplicates(winter_2021_totals, name_changes)
# Convert totals to averages 
winter_2021_averages = average_player_statistics(patched_totals)
# We can remove players that shouldnt be in this dataframewith the next helper function. 
players_to_drop = ['squishy', 'tag cramification', 'yegs', 'goofy']

TRL_W21_WK1 = remove_player_statistics(winter_2021_averages, players_to_drop)

# Calculate Relevant Stats 
offensive_df = offensive_stats(TRL_W21_WK1)
defensive_df = defensive_stats(TRL_W21_WK1)
aggression_df = aggression_stats(TRL_W21_WK1)
speed_df = speed_stats(TRL_W21_WK1)

# Create standings dataframe 
standings = generate_overall_standings(TRL_W21_WK1, offensive_df, defensive_df, aggression_df, speed_df)

In [13]:
# patched_totals.head()

In [14]:
# patched_totals.corr().head()

In [15]:
standings.head(10)

,participant,win rate,Overall,Offense,Defense,Aggression,Speed
0,invincible,0.56,100.00,86.71,24.29,100.00,91.05
1,ix mini,0.73,85.43,100.00,46.22,48.71,85.55
2,greensleeves,0.53,82.62,93.45,48.23,59.50,57.40
3,shaunch,0.47,79.49,75.95,38.30,57.97,94.32
4,rj5588,0.11,74.56,85.51,100.00,2.02,100.00
5,luisito,0.44,74.48,76.49,57.74,55.88,48.32
6,terminator,0.73,72.71,85.76,42.36,51.54,52.01
7,muffled,0.45,67.47,79.57,37.13,44.50,62.84
8,inf3ct3ds0ldi3r,0.45,65.63,83.02,55.92,32.38,50.42
9,brictone,0.11,63.96,61.33,60.98,39.00,63.65


In [ ]:
#  Deeplearning Based Overall 

In [ ]:
# TODO 
# Attempt to assign overalls to players by apply K means clustering to them to assign groups
# and then normalize to assign rank within each cluster